In [1]:
from Bio import SeqIO
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
from collections import Counter
import warnings
warnings.simplefilter("error", RuntimeWarning)
import pickle

In [2]:
cyto=list(SeqIO.parse('cyto.fasta', 'fasta'))
mito=list(SeqIO.parse('mito.fasta', 'fasta'))
nucl=list(SeqIO.parse('nucleus.fasta', 'fasta'))
secr=list(SeqIO.parse('secreted.fasta', 'fasta'))
data={'cyto':cyto, 'mito':mito, 'nucl':nucl, 'secr':secr}
mean_comp = np.load('mean_comp.npy')
with open('alphabet.txt', 'rb') as fp: alphabet=pickle.load(fp)

In [3]:
seq=list(str(cyto[0].seq))
seq_len = len(seq)
seq_counts = Counter(seq)

In [4]:
def counts(seq):
    c=Counter(seq)
    cnts=np.zeros((1,20))
    for idx, letter in enumerate(alphabet):
        cnts[:,idx]=c[letter]
    return cnts

def carr_type1(seq):
    seq_counts = Counter(seq)
    feats=np.zeros((1,20))
    var=mean_comp*(1-mean_comp)/seq_len
    for idx, letter in enumerate(alphabet):
        feats[:,idx]=seq_counts[letter]/seq_len
    feats = (feats - mean_comp) / (var**0.5)
    return feats

def carr_type2(seq):
    theo_mean=(seq_len+1)/2.
    theo_var=(seq_len+1)*(seq_len-seq_counts)/(12*seq_counts)
    return (cent_pos_mean-theo_mean)/(theo_var**0.5)

def carr_type3(seq):
    pos_vector=np.arange(1,len(seq)+1,1)
    distributional=np.zeros((1,20))
    
    for idx, letter in enumerate(alphabet):
        pos = [pos_vector[idx] for idx, aa in enumerate(seq) if aa == letter]
        if pos:
            pos=np.asarray(pos, dtype=np.float32)
            distributional[:,idx]=np.sum((pos-cent_pos_mean[:,idx])**2)
            
    measure_factor=(seq_len+1)/(seq_len*(seq_counts-1))
    measure=measure_factor*distributional
    theo_mean=(seq_len**2-1)/12.
    theo_var=(seq_len-seq_counts)*((seq_len-1)**2)*(seq_len+1)*(2*seq_counts*seq_len+3*seq_len+3*seq_counts+3)/(360*seq_counts*(seq_counts-1)*seq_len)
    return (measure-theo_mean)/(theo_var**0.5)
    
def cent_mean(seq):
    pos_vector=np.arange(1,len(seq)+1,1)
    centroidal_position=np.zeros((1,20))
    for idx, letter in enumerate(alphabet):
        pos = [pos_vector[idx] for idx, aa in enumerate(seq) if aa == letter]
        try:
            centroidal_position[:,idx] = np.mean(pos)
        except:
            centroidal_position[:,idx] = 0
    return centroidal_position

In [5]:
seq_counts=counts(seq)
cent_pos_mean=cent_mean(seq)
seq_len=len(seq)
feats1=carr_type1(seq)
feats2=carr_type2(seq)
feats3=carr_type3(seq)

In [6]:
feats1

array([[ 1.76606765, -2.12282106, -1.32271513,  0.41134867, -1.51117938,
         2.27085027, -0.23765531, -3.80841125, -1.00634121, -0.85821426,
        -0.89069634, -1.91622611,  4.10326743, -0.91257724, -0.20460399,
         2.32926458,  1.29343341,  0.79765619, -0.24170778, -0.4883382 ]])

In [7]:
feats2

array([[ 1.17056749,  0.56287188,  0.11427425, -1.06421718, -1.70480355,
        -0.95542086, -1.62896093, -0.39711415,  1.45675871,  0.49106089,
        -0.3164912 , -0.55134198,  3.2318366 ,  1.240002  , -0.85291514,
         0.38884143,  0.51804057, -0.33647561, -1.98570598, -3.14485889]])

In [8]:
feats3

array([[-0.13080281,  0.39202966,  0.27349589, -0.49944096, -1.52704365,
         0.79588087,  0.07612193,  0.75161157, -1.58916108, -0.05440881,
        -1.69016675,  0.96460931, -0.90838914,  2.43986651,  0.18311363,
        -0.12018133, -0.51858465,  1.9594124 , -1.78884142, -3.08487058]])